In [9]:
# https://github.com/Victor0118/cross_domain_embedding
class zipf(object):
    def __init__(self, ranks):
        self.ranks= ranks
        
    def frequency(self):
        #print (self.rank)        
        f= {}
        N= 0
        
        for word, rank in self.ranks:
            freq= 1/rank
            f[word]= freq
            N += freq
        
        for word in f: f[word] /= N
        #print(f)
        return f

#target
import gensim
import operator

class CorpusRank(object): # normalized 
    def __init__(self, filename):
        self.filename = filename
        self.MAX_SENTENSE_SIZE= 1000 # in general 15-20 words
        
    def iter(self):
        print("reading target domain and tokenization", self.filename)
        for sentense in open(self.filename):
            #print(sentense)
            #if len(sentense) > self.MAX_SENTENSE_SIZE:    continue
            for token in gensim.utils.tokenize(sentense, encoding='utf8',to_lower=True):    yield token 
            #yield line.strip().split(' ')
                
    def rank(self):
        dic= {}
        
        print("computing target domain rank...")
        for word in self.iter():
            #print(word)
            if word not in dic:    dic[word]= 0
            dic[word] += 1
        
        # sort by dictionary value 
        sorted_tuple = sorted(dic.items(), key=operator.itemgetter(1),reverse=True)
        
        l_rank= []
        rank= 1
        for key, val in sorted_tuple:
            l_rank.append( (key, rank) )
            rank+= 1

            
        print("numeber of tokens:",rank)
        return l_rank

#source
from gensim.models import word2vec
from gensim.models import KeyedVectors

class WvRank(object): # normalized 
    def __init__(self, filename):
        self.filename = filename
        
    def read_file(self):
        print("loading source model...")
        model= KeyedVectors.load_word2vec_format(self.filename, binary=True)
        print( self.filename + " model dimentional is", model.vector_size)
        return model
    
    def rank(self):
        dic= {}
        rank= []       
        print("computing source domain frequnecy...")
        for word, vocab_obj in self.read_file().vocab.items():   dic[word]= vocab_obj.count
        
        N= len(dic)
        print("number of dictionary:", N)
        for word in dic: rank.append( (word, N+1-dic[word]) )
        
        return rank
   

class SimilarityScore(object): 
    def __init__(self, f1, f2, file_name):
        self.f1 = f1
        self.f2 = f2
        self.file_name = file_name
    
    def save(self):
        outF = open(self.file_name, "w")
        score= {}
        for w in f1:
            if w in f2:
                score[w]= 2*f1[w]*f2[w]/(f1[w] + f2[w])
                outF.write('$'+ w +'$' + ' ' + '$'+ str(score[w])+'$')
                outF.write("\n")
        outF.close()
        print(self.file_name, "has been saved")

        
target= 'word2vecTrainCorpus.txt'
#target= 'corpus.txt'
#target= '1000000_lines.txt'

#source= 'GoogleNews-vectors-negative300.bin'        
#source= '10000_lines.bin'
source= 'wv2glove.6B.300d.bin' 

freq1= CorpusRank(target).rank()     
freq2= WvRank(source).rank() 

f1, f2= zipf(freq1).frequency(), zipf(freq2).frequency()
SimilarityScore(f1, f2, '_similarity_score.txt').save()

computing target domain rank...
reading target domain and tokenization word2vecTrainCorpus.txt
numeber of tokens: 532526
computing source domain frequnecy...
loading source model...
wv2glove.6B.300d.bin model dimentional is 300
number of dictionary: 400000
_similarity_score.txt has been saved


In [ ]:
./cre -train word2vecTrainCorpus.txt -model wv2glove.6B.300d.bin -similarity similarity_score.txt -output wv2glove.6B.300d_transfer_model.bin -size 300 -window 5 -binary 1 -lambda 10 -threads 20